In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:52:37.732823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:52:37.832740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:52:37.832756: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:52:37.853511: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:52:38.378106: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 13:52:43.066687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:52:43.067656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:52:43.067750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:52:43.067832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:52:43.067910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 394ms/step - loss: 2.7743 - auc: 0.7911
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 2.4164 - auc: 0.8245
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 2.1843 - auc: 0.8859
Epoch 4/100
1/1 [==============================] - 0s 41ms/step - loss: 1.9684 - auc: 0.9481
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8188 - auc: 0.9721
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6859 - auc: 0.9848
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5622 - auc: 0.9936
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4710 - auc: 0.9956
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3817 - auc: 0.9968
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3143 - auc: 0.9955
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2161 

1/1 [==============================] - 0s 27ms/step - loss: 0.1201 - auc: 0.9988
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1255 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1214 - auc: 0.9984
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1176 - auc: 0.9989
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1254 - auc: 0.9981
Epoch 93/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1194 - auc: 0.9987
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1231 - auc: 0.9979
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1177 - auc: 0.9986
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1200 - auc: 0.9982
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1188 - auc: 0.9987
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [3]:
name='remove_svm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.3788 - auc: 0.7242
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.4153 - auc: 0.8418
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2000 - auc: 0.8736
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.9511 - auc: 0.8932
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.7288 - auc: 0.9224
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6593 - auc: 0.9208
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5388 - auc: 0.9464
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4083 - auc: 0.9728
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3036 - auc: 0.9858
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2210 - auc: 0.9906
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.1426 - auc: 0.9980
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1438 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1410 - auc: 0.9980
Epoch 91/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1442 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1428 - auc: 0.9982
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1392 - auc: 0.9981
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1410 - auc: 0.9982
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1394 - auc: 0.9981
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1410 - auc: 0.9981
Epoch 97/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1396 - auc: 0.9980
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [4]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.4829 - auc: 0.5942
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2679 - auc: 0.8413
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 2.2778 - auc: 0.8739
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 2.1412 - auc: 0.8915
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9309 - auc: 0.9069
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.7290 - auc: 0.9241
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5842 - auc: 0.9419
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5201 - auc: 0.9443
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4765 - auc: 0.9413
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3744 - auc: 0.9615
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1573 - auc: 0.9975
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1537 - auc: 0.9975
Epoch 90/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1437 - auc: 0.9988
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1566 - auc: 0.9969
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1570 - auc: 0.9967
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1477 - auc: 0.9981
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1563 - auc: 0.9972
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1582 - auc: 0.9967
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1499 - auc: 0.9978
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1504 - auc: 0.9973
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 2.5516 - auc: 0.5801
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 2.3417 - auc: 0.8368
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2960 - auc: 0.8723
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 2.1087 - auc: 0.8972
Epoch 5/100
1/1 [==============================] - 0s 31ms/step - loss: 1.8863 - auc: 0.9184
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6979 - auc: 0.9360
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5615 - auc: 0.9566
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5267 - auc: 0.9492
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4451 - auc: 0.9594
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3451 - auc: 0.9751
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1385 - auc: 0.9984
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1412 - auc: 0.9984
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1395 - auc: 0.9980
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1394 - auc: 0.9981
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1380 - auc: 0.9984
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1351 - auc: 0.9986
Epoch 94/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1386 - auc: 0.9981
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1428 - auc: 0.9976
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1390 - auc: 0.9980
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1349 - auc: 0.9982
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [6]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 2.3365 - auc: 0.7431
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.4001 - auc: 0.8480
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 2.1586 - auc: 0.8740
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.8760 - auc: 0.9001
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7055 - auc: 0.9237
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.6168 - auc: 0.9342
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4997 - auc: 0.9584
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3752 - auc: 0.9805
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2804 - auc: 0.9879
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2090 - auc: 0.9919
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1412 - auc: 0.9979
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1417 - auc: 0.9974
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1410 - auc: 0.9973
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1360 - auc: 0.9986
Epoch 92/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1405 - auc: 0.9983
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1398 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1354 - auc: 0.9984
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1328 - auc: 0.9988
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1392 - auc: 0.9980
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1402 - auc: 0.9975
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [7]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.5241 - auc: 0.6302
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.4031 - auc: 0.8428
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 2.2738 - auc: 0.8762
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 2.0565 - auc: 0.8947
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8239 - auc: 0.9166
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6337 - auc: 0.9471
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5422 - auc: 0.9577
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4738 - auc: 0.9623
Epoch 9/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3804 - auc: 0.9752
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2688 - auc: 0.9892
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.1351 - auc: 0.9979
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1308 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1288 - auc: 0.9985
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1269 - auc: 0.9986
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1296 - auc: 0.9982
Epoch 93/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1260 - auc: 0.9987
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1265 - auc: 0.9984
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1339 - auc: 0.9975
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1279 - auc: 0.9982
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1260 - auc: 0.9983
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [8]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 2.6543 - auc: 0.5126
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.2554 - auc: 0.8481
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.2751 - auc: 0.8837
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 2.1911 - auc: 0.8987
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 2.0310 - auc: 0.9084
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.8193 - auc: 0.9276
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6513 - auc: 0.9407
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5239 - auc: 0.9590
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4795 - auc: 0.9570
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4278 - auc: 0.9577
Epoch 11/100
1/1 [==============================] - 0s 28ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1525 - auc: 0.9975
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1512 - auc: 0.9976
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1475 - auc: 0.9982
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1501 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1509 - auc: 0.9976
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1468 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1493 - auc: 0.9973
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1547 - auc: 0.9964
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1473 - auc: 0.9977
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1513 - auc: 0.9975
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [9]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('6model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.937397
1     remove_svm  0.942242
2      remove_rf  0.937991
3   remove_dmpnn  0.932851
4      remove_lr  0.931431
5    remove_fcnn  0.935496
6     remove_xgb  0.935932
